In [ ]:
# standard python utilities
import os
from os.path import dirname, basename, exists, join
import sys
import glob
import pandas as pd
import numpy as np
import calendar
import time

# standard python plotting utilities
import matplotlib as mpl
import matplotlib.pyplot as plt

# standard geospatial python utilities
import pyproj # for converting proj4string
import shapely
import geopandas as gpd
import rasterio

# mapping utilities
import contextily as ctx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm



In [ ]:
doc_dir = os.getcwd()
while basename(doc_dir) != 'Documents':
    doc_dir = dirname(doc_dir)
    
git_dir = join(doc_dir,'GitHub','CosumnesRiverRecharge')
# dir of all gwfm data
gwfm_dir = join(dirname(doc_dir),'Box/research_cosumnes/GWFlowModel')

flopy_dir = doc_dir+'/GitHub/flopy'
if flopy_dir not in sys.path:
    sys.path.insert(0, flopy_dir)

import flopy 
import flopy.utils.binaryfile as bf


In [ ]:
# set box directory for output figures and data
box_dir = gwfm_dir+'/Levee_setback/levee_setback_distance_analysis/'

fig_dir = box_dir+'figures/'
data_dir = box_dir+'data_output/'

chan_dir = box_dir+'channel_data/'
gis_dir = chan_dir+'GIS/'

In [ ]:
ext_dir = 'F:/WRDAPP'
c_dir = 'C:/WRDAPP'

if os.path.exists(ext_dir):
    loadpth = ext_dir 
elif os.path.exists(c_dir):
    loadpth = c_dir 

loadpth = loadpth +'/GWFlowModel/Cosumnes/levee_setback/'
model_ws = loadpth+'flood_depth_analysis'

In [ ]:
nrow = 100
ncol = 230
nlay = 1
delr = 200
delc = 200



In [ ]:
# burn river shapefile into the 10 meter dem and then read it out to find the cells where it is
# from flopy.utils import Raster
# # Full size dem of northern sac valley
raster_name = gwfm_dir+"/DEM_data/USGS_ten_meter_dem/modeldomain_10m_transformed.tif"

# # rio10_utm = Raster.load(raster_name)
# rio10_utm = rasterio.open(raster_name)
# dem_10m = rio10_utm.read((1,))

In [ ]:
# grid_sfr = gpd.read_file(gwfm_dir+'/SFR_data/final_grid_sfr/grid_sfr.shp')
grid_p = gpd.read_file(gwfm_dir+'/DIS_data/grid/grid.shp')

m_domain = gpd.read_file(gwfm_dir+'/DIS_data/NewModelDomain/GWModelDomain_52_9deg_UTM10N_WGS84.shp')
grid_sfr = gpd.read_file(gwfm_dir+'/SFR_data/final_grid_sfr/grid_sfr.shp')
# load sacramento river, creeks
rivers = gpd.read_file(gwfm_dir+'/SFR_data/Sac_valley_rivers/Sac_valley_rivers.shp')
cr = gpd.overlay(rivers.loc[rivers.GNIS_Name=='Cosumnes River'].to_crs('epsg:32610'), m_domain)

In [ ]:
from shapely.geometry import MultiLineString, LineString, Point, shape, mapping
from shapely.ops import linemerge
import fiona

cr_line = MultiLineString(cr.geometry.values)

In [ ]:
import h5py
f = h5py.File(join(chan_dir, 'setback_locs.hdf5'), "r")
local_str_setbacks = f['setbacks']['local'][:]
str_setbacks = f['setbacks']['regional'][:]

f.close()

### Parallel XS

In [ ]:


geom = shape(cr_line)

# # length of the LineString
length = int(geom.length)
sfr_sp = gpd.GeoDataFrame(pd.DataFrame(np.zeros((length,1)),columns=['id']))
sfr_sp['geometry'] = shapely.geometry.Point(0,0)
# enumerate keeps track of the count in addition to the object being iterated on
for i, distance in enumerate(range(0, length, 1)):
    point = geom.interpolate(distance)
    sfr_sp.loc[i,'geometry'] = point
    sfr_sp.loc[i,'id'] = i

In [ ]:
sfr_sp['Easting'] = sfr_sp.geometry.x.values
sfr_sp['Northing'] = sfr_sp.geometry.y.values

point = sfr_sp.loc[:,['Easting','Northing']].values

with rasterio.open(raster_name) as src:
    sfr_sp['z_ft'] = [sample[0] for sample in src.sample(point)]
    
sfr_sp['z_m'] = sfr_sp.loc[:,'z_ft']*0.3048


Find XS every 1000 meters avoids too much overlap in XS,  after calculating depth there is some discontinuity but WSE is uniform with slope, so the numebr could be reduced to 2000m which would also aid muskingum-cunge routing requirements (coarser time step allowed).   
I also realized that I should simplify the river feature to linearize it and avoid XS occuring on oddities where there is a large turn in the channel for a short distance. Adding a 500m tolerance simplify additionally reduces overlap and produces good XS locations/angles.

In [ ]:

def make_transects(geom, dline = 2000, xs_width = 3305*2):
    """ 
    Function to make regular transects for a line object (geom : shapely polyline) 
    at some longitudinal distance (dline : distance in meters)
    max width needed is 3200 m +100m from original channel + 5 to allow full 3200 both direction
    """
    # tolerance is distance which simplified points can be from the original
    geom = geom.simplify(500)

    # how often to interpolate a point, 200 m matches model grid
    dline_long = np.copy(dline)
    # # length of the LineString
    length = int(geom.length)

    num_xs = np.floor(length/dline).astype(int)
    transects = pd.DataFrame(np.zeros((num_xs,1)), columns = ['line'])
    transects['geometry'] = LineString([(0,0),(0,1)]) #initiate LineString geometry column


    for i, distance in enumerate(range(0, int(length), dline)):
        short_line = LineString([geom.interpolate(distance),geom.interpolate(distance+dline)])
        geom_left = short_line.parallel_offset(xs_width/2,'left', resolution = 32, join_style = 2)
        geom_right = short_line.parallel_offset(xs_width/2,'right', resolution = 32, join_style = 2)
        perp_line = LineString([list(geom_left.coords)[0], list(geom_right.coords)[0]])
        transects.loc[i,'geometry'] = perp_line

    # save output
    transg = gpd.GeoDataFrame(transects)
    transg = transg.drop_duplicates('geometry')
    transg['line'] = np.arange(0,len(transg))
    transg.crs = 'epsg:32610'
    return(transg)


In [ ]:
# linemerge is essential because it joins linesegments by matching (x,y) so the line is continuous
geom = linemerge(cr.geometry.unary_union)

In [ ]:
xs_width = 3305*2
transg200 = make_transects(geom = linemerge(cr.geometry.unary_union), dline = 200, xs_width = xs_width)
transg200.to_file(gis_dir+'/transect_lines_subsegments.shp')

transg = make_transects(geom = linemerge(cr.geometry.unary_union), dline = 2000, xs_width = xs_width)
transg.to_file(gis_dir+'/transect_lines.shp')
# check cross section lines are paralle
fig,ax=plt.subplots(figsize=(6,6))
transg200.plot(color='red', ax=ax)
transg.plot(ax=ax)
cr.plot(ax=ax)


In [ ]:
def transect2pts(transg, dline = 10, xs_width = 3305*2):
    """ 
    Convert line transects (transg : geodataframe) to points 
    at a regular interval along the transect (dline : distance in meters)
    """

    xs_all = gpd.GeoDataFrame(pd.DataFrame(columns=['xs_num','dist_from_right_m','geometry']))

    for j in np.arange(0,len(transg)):
        xs = gpd.GeoDataFrame(pd.DataFrame(np.zeros((int(xs_width/dline),2)), columns=['xs_num','dist_from_right_m']))
        xs['geometry'] = Point([(0,0)])
        xs['xs_num'] = j

        # pick one geometry at a time
        geom = transg.iloc[j].geometry

        # # # length of the LineString
        length = int(geom.length)
        # create discrete points for each lien
        for i, distance in enumerate(range(0, int(length), dline)):
            point = geom.interpolate(distance)
            xs.loc[i,'geometry'] = point
            xs.loc[i,'dist_from_right_m'] = i
        # append individual cross section to all dataframe
    #     xs_all = xs_all.append(xs)
        xs_all = pd.concat([xs_all, xs], axis=0, join='outer', ignore_index=True)
    # remove na values
    xs_all = xs_all.dropna(subset=['xs_num'])
    return(xs_all)
xs_all = transect2pts(transg, dline = 10, xs_width = 3305*2)

Sample DEM for every XS point for an elevation

In [ ]:
def sample_pts(xs_all, raster_name):
    """
    Given a geodataframe of points sample elevations from a raster
    """
    src = rasterio.open(raster_name)
    xmin, xmax = src.bounds[0], src.bounds[2]
    ymin, ymax = src.bounds[1], src.bounds[3]

    xs_all.crs='epsg:32610'
    xs_all['Easting'] = xs_all.geometry.x
    xs_all['Northing'] = xs_all.geometry.y
    # filter to points that truly overlap the DEM
    xs_all = xs_all.cx[xmin:xmax, ymin:ymax]

    # point = xs_all.loc[:,['Easting','Northing']].values # old
    point = list(zip(xs_all.geometry.x, xs_all.geometry.y)) # same error

    print(os.path.basename(raster_name))
    with rasterio.open(raster_name) as src:
        xs_all['z_m'] = [sample[0] for sample in list(src.sample(point))]

    # convert distance from right from 1/10 of meters because points were every 10 meters
    # the distance was set with the index but really it should be 3200 meters not just 320 meters
    xs_all.dist_from_right_m *= 10
    # remove any NA values picked up from DEM raster
    xs_all.loc[xs_all['z_m'] == src.meta['nodata'], ['z_m']] = np.nan
    xs_all.index = np.arange(0,len(xs_all))
    return(xs_all)
xs_all = sample_pts(xs_all, raster_name)

## XS Cleaning
There isn't going to be a clear way to distinguish the levees and the XS shows bumpiness. The best solution will be to assume that levee setback will involve a cut and fill approach such that the ground surface elevation after setback is the mean of the elevation from before, but then this won't allow variable flooding based on elevation...

It will take looking at the XS in different regions.
It might be doable to fill in the channel as Sierra suggested which would raise the river above the levee and then install new "levee walls" at the desired distance. Filling in just the channel requires: 1. Going to the center line 2. go out some distance to account for channel width 3. set those values as a fraction of the levee height to insure more overbank flooding

In [ ]:
# calculate elevations for subsegments ( 200 m)
xs_all200 = transect2pts(transg200, dline = 10, xs_width = 3305*2)
xs_all200 = sample_pts(xs_all200, raster_name)

xs_all200.drop(['geometry'],axis=1).to_csv(chan_dir+'XS_point_subsegments_elevations.csv')

# convert to dataframe for easier plotting
xs_all_df200 = pd.DataFrame(xs_all200)
# pivot based on XS number and save only elevation in z_m
xs_all_df200 = xs_all_df200.pivot_table(index='dist_from_right_m',columns='xs_num',values='z_m')
xs_all_df200.to_csv(chan_dir+'Elevation_by_XS_number_subsegments_meters.csv')

In [ ]:

xs_all.drop(['geometry'],axis=1).to_csv(chan_dir+'XS_point_elevations.csv')

# convert to dataframe for easier plotting
xs_all_df = pd.DataFrame(xs_all)
# pivot based on XS number and save only elevation in z_m
xs_all_df = xs_all_df.pivot_table(index='dist_from_right_m',columns='xs_num',values='z_m')
xs_all_df.to_csv(chan_dir+'Elevation_by_XS_number_meters.csv')

In [ ]:

xs_all_df.plot()
plt.legend(ncol=4, loc=(1,0.01), title='XS Number')


The process will be too complicated too directly edit the raster, so individual cross-sections will be edited by applying a rolling mean in the floodplain in the 200-400 m around the main channel to "flatten" any levee like structure. Then new 'levee walls' will be placed at the desired setback distance.

Helen suggested looking at the historical 100 year flood to estimate the levee height they would have designed for, or look at the physical characteristics to design it. Levee construction began in 1930s, biggest flood was in 1907 at 71,000 cfs (estiamted) with a gage height of 16 ft at Michigan Bar so levee's would most likely be designed to at least 16 ft.

For each levee setback we should imagine that the channel is partially filled in, but rather than uniformly removing all levee walls within the floodplain, we breach the levee wall in 200 m sections for every 2km to allow flood water to inundate the floodplain and then new levees are constructed at the specified distance to bring ground level to 16 ft above channel bottom. In real life this height could probably be steadily lowered as there is so much room in the floodplain to accomdate the volume that a lower levee height suffices.
1. Rolling mean is used to fill in channel with earth from levee or nearby.
2. New levees are continuously installed at specified distance


It doesn't look like I will see much floodplain inundation, but I won't know until I estimate depth. Start with 2-3,000 is 1 year flood and 20,000 cfs is 5 year. The floodplain connection flow is estimated to be 2-3,000 cfs by Whipple in one paper on floodplain inundation at Oneto-Denier, and as 23 cms or 800 cfs in the flood typology paper.

In [ ]:
xs_all = pd.read_csv(chan_dir+'XS_point_elevations.csv')

xs_all_df = pd.read_csv(chan_dir+'Elevation_by_XS_number_meters.csv',index_col='dist_from_right_m')

In [ ]:
def xs_channel_smooth(xs_all_df, roll_window = 400, dline = 10):
    """ 
    Apply a rolling mean to center window (roll_window) of channel (xs_all_df) 
    to represent incision infill and reduction of bank/levee height,
    40 observations is equal to 400 meters
    """
    xs_roll_mean = xs_all_df.rolling(int(roll_window/dline), center=True).mean()
    channel_middle = int(len(xs_all_df)/2)*dline
    channel_bool = (xs_all_df.index >= channel_middle - (roll_window/2))& (xs_all_df.index <= channel_middle + (roll_window/2))
    xs_all_df.loc[channel_bool,:] = xs_roll_mean.loc[channel_bool,:]
    return(xs_all_df)


In [ ]:
xs_levee_smooth200 = xs_channel_smooth(xs_all_df200, roll_window = 400, dline = 10)
# save for flow-recharge notebook
# xs_levee_smooth200.to_csv(chan_dir+'xs_levee_smooth_subsegments.csv')


xs_levee_smooth = xs_channel_smooth(xs_all_df, roll_window = 400, dline = 10)
# # save for flow-recharge notebook
# xs_levee_smooth.to_csv(chan_dir+'xs_levee_smooth.csv')

In [ ]:
fig,ax = plt.subplots(figsize=(12,6))

xs_levee_smooth.plot(ax=ax)
ax.legend(ncol=4, loc=(1,0.01), title='XS Number')


## How to update XS to include changes in levees
The goal here is to provide a more physical representation of XS that just stopping the XS calculation in Manning's. Here the wetted perimiter of the channel would be larger at deeper depths.

This was implemented as a function called xs_setback

In [ ]:
xs_test = xs_levee_smooth.copy()
# rather than saving a version of the xs for each setback just add this adjustment
n=5
# the channel should fall within the center 400m so that minimum can be used to set new levee height
thalweg = xs_test.loc[channel_middle-roll_window/2:channel_middle+roll_window/2].min()
# check to see XS elevation at setback distance to determine if it should be raised to needed levee height
levee_diff = xs_test.loc[3100-setbacks[n],:] - thalweg
# where XS height is less than 20 ft above channel bottom then raise to 20 ft above
xs_test.loc[3100-setbacks[n],:][levee_diff< 20*0.3048] = thalweg + 16*0.3048
levee_diff = xs_test.loc[3300+setbacks[n],:] - thalweg
xs_test.loc[3300+setbacks[n],:][levee_diff< 20*0.3048] = thalweg + 16*0.3048

xs_test.iloc[:,24].plot()


In [ ]:
fig,ax=plt.subplots(figsize=(6,3))
xs_levee_smooth.min().plot(ax=ax, label='Adj XS')
xs_all_df.min().plot(ax=ax, color='red',label='XS')
grid_sfr.assign(river_km = np.cumsum(grid_sfr.length_m/dline_long)).plot(x='river_km',y='z',kind='line',ax=ax, label='Model')
plt.legend()

In [ ]:
# dem data for cropping above land surface
dem_data = np.loadtxt(gwfm_dir+'/DIS_data/dem_52_9_200m_linear.tsv')

# Mannings equation
The assumption of levee setback is that as the setback distance increases so does the wetted xs perimeter which increases the hydraulic radius (there is more roughness impacting the water) so the velocity will be lower. This analysis is independent of varying geology, and river reach under the assumption that any recontoured reach would undergo restoration resulting in similar reach rougnesses of the floodplain (small bushes, trees). And it will assume the existing average bed slope of the lower Cosumnes river applies.  
This analysis could be done by running a HEC-RAS model at all of these different setback distances, but here the goal is to determine the trend and general distance at which diminishing returns occur.  The analysis will be completed first for a typical winter flood event of 2,000 cfs which is near floodplain connectivity (need to approximate area and velocity of channel as starting point or else will need to cycle through several areas/wetted perimeters - better to just use multiple areas that connect to multiple possibles flood sizes )  
The width will range from 0 to 3200 meters in both direction but I coud have depth be dependent on the actual elevations  
Manning Equation for SI units  
$ Q = VA = (\frac{1.00}{n}) A  {R}^{2/3} \sqrt{S} $  
$ R = \frac{A}{Wp}$  
hydraulic radius r is XS area divided by wetted perimeter  
[Manning's eqn reference](http://www.fsl.orst.edu/geowater/FX3/help/8_Hydraulic_Reference/Manning_s_Equation.htm)  
Avg manning's n for in channel is 0.048  - clean, winding, some pools and shoals, but some weeds and stones, lower stages, more ineffective, slopes and sections
Floodplains have much broader range, due to variability in vegetation type: tall grass is 0.035 but dense willows is 0.15  
[Manning's n reference](http://www.fsl.orst.edu/geowater/FX3/help/8_Hydraulic_Reference/Mannings_n_Tables.htm)  

Slope for Cosumnes could be considered the 0.0006 which is the mean for all reaches

In [ ]:
fxn_dir = git_dir+'/python_utilities'
if fxn_dir not in sys.path:
    sys.path.append(fxn_dir)
# sys.path
# import muskingum_recharge as mr

from importlib import reload
# reload(mr)

In [ ]:
from muskingum_recharge import min_Q, mannings, calc_depth_arr, gridded_interpolation, xs_setback, mannings_v


In [ ]:
# grid x,y coords for interpolation output
# x_out = grid_p.geometry.centroid.x.values
# y_out = grid_p.geometry.centroid.y.values



## Return period floods (5, 20, 50,100 yr)

In [ ]:
peak_flows = pd.read_csv(box_dir+'michigan_bar_peak_flows.tsv', delimiter='\t', comment='#')
peak_flows = peak_flows.drop(0)
peak_flows = peak_flows.dropna(subset=['peak_va'])
peak_flows = peak_flows[['peak_dt','peak_va','gage_ht']]
peak_flows.peak_va = pd.to_numeric(peak_flows.peak_va)
annual_max_df_sorted = peak_flows.sort_values('peak_va', ascending=True)

# code from hydro-informatics exercsises
n = annual_max_df_sorted.shape[0]
annual_max_df_sorted.insert(0, "rank", range(1, 1 + n))
annual_max_df_sorted["pr"] = (n - annual_max_df_sorted["rank"] + 1) / (n + 1)
annual_max_df_sorted["return-period"] = 1 / annual_max_df_sorted["pr"]
annual_max_df_sorted["period_int"] = annual_max_df_sorted["return-period"].astype(int)

# annual_max_df_sorted[annual_max_df_sorted.period_int==50]

# identify relevant return period flows
per_id = []
for per in [1,2, 5, 10, 20, 50, 100]:
    per_id = per_id + [np.argmin(np.abs((annual_max_df_sorted['return-period']-per).values))]
per_table = annual_max_df_sorted.iloc[per_id][['peak_dt','return-period','peak_va']].round(2)
per_table = per_table.rename(columns={'peak_va':'Discharge (cfs)'})

annual_max_df_sorted.plot(x='return-period',y='peak_va', figsize=(6,3), legend=False)

display(per_table)

In [ ]:
# quick calculations for the publication on flow regime
# 93000*(0.3048**3)
# slope = (49700-37500)/(57.50-19.17)
# ((20-19.17)*slope + 37500)*(0.3048**3)
# # 22500*(0.3048**3)
# 400*(0.3048**3)

In [ ]:
# from scipy.optimize import fsolve, least_squares, minimize, minimize_scalar

# fsolve(mannings, [2], args = (Q_cms, xs_elevs), xtol=1E-5)
# least_squares(mannings, [2], args = (Q_cms, xs_elevs), bounds = (0, 10), ftol=1E-5)
# minimize(mannings, [1.], args = (Q_cms, xs_elevs), bounds=[(0,10)], tol=1E-5)

# # res = minimize_scalar(mannings,  args = (Q_cms, xs_elevs), bounds=(0,10), method='bounded')
# res

# Iterate across all levee setbacks here
Checking issue where inundated area seems to decrease in some areas when setback is increased. it's okay for inundated area to decrease because when setback increases the depth is decreased and so there will be some cells that are now above the flood

In [ ]:
mb_daily = pd.read_csv(gwfm_dir+'/SFR_data/MB_daily_flow_cfs_2010_2019.csv').dropna()

In [ ]:
# mb_daily.plo/t(x='datetime',y='flow_cfs', legend=False)
# mb_daily[(mb_daily.flow_cfs > 20000)].plot(x='datetime',y='flow_cfs')

In [ ]:
# cms to cmh sustained for about 0.6 hour based on how long flood wave travels 2 km
# 
# total_vol = 40*60*60*0.6
total_vol = 40*60*60*24*7 # if flow occurred for 7 days

# calculate area for setbacks
max_area = str_setbacks[-1,:,:].sum()*200*200
# area from high flow cells within 3000m setback
max_area = 16640000.0

# even if we had that recharge rate for a full week, theoretically we only recharge 0.07 m of water
# if we limit the recharge volume to only those local cells then we would only see 1.45 m distributed

total_vol/max_area

In [ ]:
mb_daily['flow_cms'] = mb_daily.flow_cfs * (0.3048**3)
mb_daily.plot(x='datetime',y='flow_cms', legend=False)
plt.ylabel('Flow (cms)')

In [ ]:
# n=3
# mb_daily[mb_daily.flow_cfs > per_table['Discharge (cfs)'].iloc[n]]
# mb_daily[mb_daily.flow_cfs > 2000]

In [ ]:
# Q_cfs = 93000 #  115 year
# Q_cfs = 39400 # 23 year return period
Q_cfs = 22500 # 5 year
# Q_cfs = 2000 # 1ish year

Q_cms = Q_cfs*(0.3048**3) # convert to cubic meters per second



 will need to iterative solve for Area, area is the sum of area between depth and ground surface
 
 As we iterate down the XS we need to account for water lost to recharge

In [ ]:
# find minimum from channel center
xs_mins = xs_levee_smooth.loc[3100:3300].min(axis=0)
xs_mins.index = xs_mins.index.astype(int)
# xs_mins.interpolate(method='linear').plot()
slope = xs_mins.diff().rolling(2, center=True, closed='right').mean().bfill()/2000*-1
adj_xs_mins = np.append(xs_mins[0], (xs_mins[0]-slope.cumsum()*2000))


In [ ]:
nseg = 20
setback = 8*200
xs_elevs = xs_setback(xs_levee_smooth.iloc[:,nseg].copy(), setback, 30)

def depth_match(xs_elevs, nseg, setback):
    """ For a XS find the depth that best matches the flow given with stepwise global sampling"""
    q_test = 0
    dec = 10
    di=1
    df = 1E-6
    # 3 decimal places is enough to be within 1 cms
    for dec in [10, 1, 0.1, 0.01]:
        # set range of values to test above last value if too low or below if too high
        if q_test < Q_cms:
            d_test = np.arange(df, df + dec, dec/10)
        else:
            d_test = np.arange(df - dec, df, dec/10)
        # calculate flow for depth range
        q_test = [mannings([d], xs_elevs, n, slope.iloc[nseg]) for d in d_test]
        print(df,q_test)
        # find depth with minimum error
        di = np.argmin(np.abs(np.subtract(q_test, Q_cms)))
        q_test = q_test[di]
        # add the decimal place to the depth
        df += np.copy(di)*dec/10
    return(df)

d_out = depth_match(xs_elevs, nseg, setback)
d_out, mannings([d_out], xs_elevs, n, slope.iloc[nseg])

In [ ]:
from scipy.optimize import minimize_scalar, minimize
from scipy.optimize import basinhopping

n= 0.048

# setback = 3000
# depths = np.zeros(xs_levee_smooth.shape[1])
depths = np.zeros((len(setbacks), xs_levee_smooth.shape[1]))
q_check = np.zeros(depths.shape)

# iterate across all setbacks
for nseg in np.arange(0, xs_levee_smooth.shape[1]): #xs_levee_smooth.shape[1]
    # setbacks don't go past 3000, enumerate includes 3200 which we don't want
    for s,setback in enumerate(setbacks): 
        # for a given setback imagine there is an impenetrable levee blocking overbank flow
#         xs_elevs = xs_levee_smooth.iloc[:,nseg][3100-setback:3300+setback]
        xs_elevs = xs_setback(xs_levee_smooth.iloc[:,nseg].copy(), setback)
    
        # solve for depth that matches given flow
        res = minimize_scalar(min_Q, args = (xs_elevs, n, slope.iloc[nseg], Q_cms), 
                                  bounds=(0.1,10), method='Golden', tol=1E-3)
        if res.fun>0.05*Q_cms: # greater than 5% difference try to fix with bounded solving
            print(str(nseg),' ', s, '%.2f'%res.fun, 'iter %i'%res.nit, ', ',res.success, 'd %.2f'%res.x)
        if mannings(res.x, xs_elevs, n, slope.iloc[nseg]) > Q_cms*1.05:
            res = minimize_scalar(min_Q, args = (xs_elevs, n, slope.iloc[nseg], Q_cms), 
                          bounds=(0.1,res.x-0.1), method='bounded', tol=1E-3)
        if mannings(res.x, xs_elevs, n, slope.iloc[nseg]) < Q_cms*0.95:
            res = minimize_scalar(min_Q, args = (xs_elevs, n, slope.iloc[nseg], Q_cms), 
                          bounds=(res.x+0.1,10), method='bounded', tol=1E-3)
        # changing tolerance doesn't improve function error
        depths[s, nseg] = res.x
#         depths[s,nseg] = depth_match(xs_elevs, nseg, setback)
        q_check[s,nseg] = mannings(depths[s,nseg], xs_elevs, n, slope.iloc[nseg])

In [ ]:
d_arr = np.zeros((len(setbacks), nrow, ncol))
for s,setback in enumerate(setbacks):
# s, setback = 2, 400
#     wse_all = pd.DataFrame(xs_levee_smooth[3100-setback:3300+setback].min()+depths[s,:], columns=['wse_m'])
    wse_all = pd.DataFrame(xs_levee_smooth.min()+depths[s,:], columns=['wse_m'])
    d_arr[s,:,:] = calc_depth_arr(wse_grid, wse_all)
    
d_arr_out = np.reshape(d_arr, (len(setbacks)*nrow, ncol))
np.savetxt(box_dir+'channel_data/depth_array_'+str(Q_cfs)+'cfs.tsv', d_arr_out, delimiter='\t')


In [ ]:

# s1 = 1
# s2 = 2
# print('Flood cells matching in setback ',s1,' and ',s2,' :',((d_arr[s2,:]>0) &(d_arr[s1,:]>0)).sum(), 
#       'flood cells in setback ',s1,':', (d_arr[s1,:]>0).sum())
# # plt.imshow(d_arr[1,:])
# # plt.show()
# plt.imshow(d_arr[s1,:])
# plt.show()
# plt.imshow(d_arr[s2,:])
# plt.show()
# plt.imshow(d_arr[s1,:]*((d_arr[s2,:]>0) &(d_arr[s1,:]>0)))

In [ ]:
V_arr = np.zeros((len(setbacks), xs_levee_smooth.shape[1]))
for s,setback in enumerate(setbacks):
    for nseg in np.arange(0,xs_levee_smooth.shape[1]):
        # for a given setback imagine there is an impenetrable levee blocking overbank flow
        xs_elevs = xs_setback(xs_levee_smooth.iloc[:,nseg].copy(), setback, 30)
        # solve for depth that matches given flow
        V_arr[s, nseg] = mannings_v(depths[s, nseg], 0, xs_elevs) # output is Q_calc - Q so if Q=0 then output is Q_calc


In [ ]:
fig,ax = plt.subplots(1,2, figsize=(10,3))
ax_n= ax[0]
pd.DataFrame(V_arr,index=setbacks).plot(cmap='viridis', legend=False, ax = ax_n)
ax_n.set_xlabel('Setback distance (m)')
ax_n.set_ylabel('Manning Eqn\nVelocity (m/s)')

ax_n= ax[1]
pd.DataFrame(V_arr[0,:]/V_arr,index=setbacks).plot(cmap='viridis', legend=False, ax = ax_n)
ax_n.set_xlabel('Setback distance (m)')
ax_n.set_ylabel('Times reduction \nin velocity \n(increase in travel time)')

fig.tight_layout()

In [ ]:
fig,ax=plt.subplots(2,1, figsize=(8,4),sharex=True)
ax_n = ax[0]
wse_plt = np.zeros(depths.shape)
for s,setback in enumerate(setbacks):
    wse_plt[s,:] = (xs_levee_smooth[3100-setback:3300+setback].min()+depths[s,:])
#     (xs_levee_smooth[3100-setback:3300+setback].min()+depths[s,:]).plot(ax=ax_n,  label=setback)

pd.DataFrame(wse_plt,index=setbacks).transpose().plot(cmap='viridis',legend=False, ax = ax_n)

# ax_n.plot(xs_levee_smooth.min()+depths)
# ax_n.set_xlabel('XS Number - Upstream to Downstream')
ax_n.set_ylabel(' Water Surface\nElevation (m)')
ax_n.legend(ncol=4)

print(' %.5F is the slope' % (15/(62*1000)), 'which matches the mean slope of the streambed')

ax_n = ax[1]
# ax_n.plot(depths)
pd.DataFrame(depths,index=setbacks).transpose().plot(cmap='viridis',legend=False, ax = ax_n)

# for s,setback in enumerate(setbacks):
#     ax_n.plot(depths[s,:], cmap='magma')
ax_n.set_xlabel('XS Number - Upstream to Downstream')
ax_n.set_ylabel('Depth (m)')

# plt.savefig(fig_dir+ 'water surface elevation-depth profile (m) at' + str(Q_cfs)+'cfs.png')


In [ ]:
# plot different setback XS for middle segment
plt_xs = np.arange(0,setbacks.shape[0],2)
ny = 3
nx = int(np.ceil(plt_xs.shape[0]/ny))

fig,ax = plt.subplots(nx,ny, figsize=(ny*4,nx*2),sharex=True, sharey=True)

nxs = int(xs_levee_smooth.shape[1]/2)
nxs= 20

for n, s in enumerate(plt_xs):
    setback = setbacks[s]
    ax_n = ax[int(n / ny), n % ny] if (nx > 1) else ax[i]
    xs_elevs = xs_setback(xs_levee_smooth.iloc[:,nxs].copy(), setback=setback)
    # plot levee setback adjusted XS
    xs_elevs.plot(ax=ax_n)
    # plot original cross-section    
    mid = np.mean(xs_levee_smooth.index) # location that should be channel bottom based on NHD line

    xs_all_df.iloc[:,nxs][mid-100-setback:mid+100+setback].plot(ax=ax_n)
    depths_plt = depths[s,:]
    ws = xs_elevs.min() +depths_plt[nxs]
    ax_n.plot([mid-100-setback, mid+100+setback],[ws,ws], label='setback')

    ax_n.set_title(str(setback))#+' , '+ str(np.round(ws,2))
    ax_n.set_xlabel('')
    xs_A = (ws - xs_elevs)[(ws - xs_elevs)>0].sum()*10
    print('XS '+str(setback)+' area: %.2f' %xs_A, 'm^2',end=',')
    print('V %.2f' %V_arr[s, nxs], 'Q %.2f' %(V_arr[s,nxs]*xs_A) )
#         n+=1

fig.tight_layout()
fig.text(-0.01, 0.4, 'Elevation (m)', rotation=90, size=16)
fig.text(0.4, -0.03, 'Distance from right (m)', size=16)

# plt.savefig(fig_dir +str(setback)+'setback_XS_with_depth_' +str(Q_cfs)+ 'cfs.png')

In [ ]:
# depths_plt = depths[0,:]

# nx,ny = 12,5
plt_xs = np.arange(0,xs_levee_smooth.shape[1],4)
ny = 4
nx = int(np.ceil(plt_xs.shape[0]/ny))
setback = 3200
s=16
fig,ax = plt.subplots(nx,ny, figsize=(ny*4,nx*2),sharex=True, sharey=False)
n=0
for n,nxs in enumerate(plt_xs):
# for i in np.arange(0,nx):
#     for j in np.arange(0,ny):
#         ax_n = ax[i,j]
    ax_n = ax[int(n / ny), n % ny] if (nx > 1) else ax[i]
    xs_elevs = xs_setback(xs_levee_smooth.iloc[:,nxs].copy(), setback=setback)
    xs_elevs.plot(ax=ax_n)    
    mid = np.mean(xs_levee_smooth.index) # location that should be channel bottom based on NHD line
    xs_all_df.iloc[:,nxs][mid-100-setback:mid+100+setback].plot(ax=ax_n)
    for s,setback in enumerate(setbacks):
        depths_plt = depths[s,:]
        ws = xs_levee_smooth.iloc[:,nxs][3100-setback:3300+setback].min() +depths_plt[nxs]
        ax_n.plot([mid-100-setback, mid+100+setback],[ws,ws], label='setback')

    ax_n.set_title(nxs)
#         n+=1
fig.tight_layout()
# plt.savefig(fig_dir +str(setback)+'setback_XS_with_depth_' +str(Q_cfs)+ 'cfs.png')